<div align="center">

# Обзор научных работ по StyleGAN

</div>

В этом обзоре я рассмотрю две смежные работы, предлагающих методы обучения нейронной сети  
StyleGAN, представленной в [Analyzing and Improving the Image Quality of StyleGAN](https://arxiv.org/abs/1912.04958), для генерации и  
редактирования изображений с помощью текста. Они опираются на использование модели  
CLIP ([Learning Transferable Visual Models From Natural Language Supervision](https://arxiv.org/abs/2103.00020)) от OpenAI, позволяющей  
отображать изображения и текстовые описания в одно семантически связное линейное пространство.

## StyleCLIP
Ссылка на статью: https://arxiv.org/abs/2103.17249

Авторы работы предлагают три метода редактирования изображений:
1. **Latent Optimization**:  
    - Оптимизация в пространстве $\mathcal{W}^+$ для минимизации составного CLIP-loss  
    - Тензор $\mathcal{W}^+$ состоит из 18 копий базового вектора $\mathcal{W}$ размерности 512
    - Метод не требует предварительного обучения, но вывод модели (этап оптимизации латента) долгий –  
      в работе указано 98с для NVIDIA GTX 1080Ti на 200-300 итерациях
    - На практике ожидается подбор коэффициентов функции потерь вручную в каждом отдельном случае,  
      т.к. их оптимальный выбор зависит от целевого изображения и характера изменений

2. **Latent Mapper**:  
    - Обучение маппера $M: \mathcal{W}^+ \rightarrow \mathcal{W}^+$
    - Маппер состоит из трёх полносвязных сетей, каждая из которых преобразуют часть набора  
      входных латентов, отвечающую в генераторе за свой уровень деталей изображения
    - В зависимости от задачи и промпта, может быть достаточно обучить только часть маппера
    - Применяется для конкретного типа манипуляции
    - Более быстрый на этапе вывода подход

3. **Global Directions**:  
    - Поиск глобальных направлений в $\mathcal{W}^+$ и в итоге – в $\mathcal{S}$
    - Линейная интерполяция вдоль этих направлений
    - Адаптированная версия этого подхода применяется в StyleGAN-NADA

### Векторы и их значение:
- $t$ (text) - эмбеддинг целевого промпта в текстовом подпространстве CLIP $\mathcal{T}$
- $\Delta t$ - разница между эмбеддингами целевого и исходного описаний
- $i$ (image) - эмбеддинг исходного изображения в подпространстве CLIP для изображений $\mathcal{I}$
- $\Delta i$ - разница между эмбеддингами целевого и исходного изображений в $\mathcal{I}$
- $s$ - "style code", набор параметров слоёв генератора для исходного изображения в стилевом  
    пространстве StyleGAN – $\mathcal{S}$
- $\Delta s$ - искомое изменение этого объекта для изображения в процессе оптимизации

Технически основной задачей метода **Global Directions** является отображение вектора $\Delta t$ в $\Delta s$ для  
создания целевого изображения $G(s + \alpha\Delta s)$, где $\alpha$ – коэффициент скорости движения в нужную  сторону  
в стилевом пространстве $\mathcal{S}$.

$\Delta t$ формируется следующим образом:
1. С помощью **ImageNet prompt bank** формируется определённое кол-во пропмтов описания исходного  
    изображения, затем их CLIP-эмбеддинги усредняются для более стабильного результата
2. То же применяется к целевому промпту
3. Нормализованная разница между получившимися векторами и есть $\Delta t$

Далее вычисляется, как поэлементое изменение вектора $s$ влияет на изменение эмбеддинга  
изображения $\Delta i$ (коллинеарного $\Delta t)$, в итоге формируется искомый $\Delta s$.

</br>

## StyleGAN-NADA
Ссылка на статью: https://arxiv.org/abs/2108.00946

Цель у StyleGAN-NADA та же, что у StyleCLIP – реализовать метод редактирования изображений исключительно  
с помощью текста, используя предобученные модели StyleGAN и CLIP. Основная идея – в дообучении генератора  
StyleGAN, ориентируясь на вывод CLIP в ходе оптимизации подмножества параметров модели.

### Выбор функционала потерь

В начале обсуждения своего метода авторы размышляют о наивной функции потерь, использующей косинусное  
расстояние между эмбеддингами изображения и текста в пространстве CLIP:
$$
\mathcal{L}_{global} = D_{CLIP}\big[G(w), t_{target}\big]
$$

Они приходят в выводу, что такой лосс в чистом виде мало применим в качестве основного в силу склонности  
получаемой модели к артефактам и однородныму выводу (т.н. mode collapse). Однако $D_{CLIP}$ всё же используется  
в работе далее – для выбора обучаемых слоёв генератора.

Далее авторы черпают вдохновение в третьем методе StyleCLIP, который заключается в поиске направления от  
источника к цели в пространстве CLIP. Теперь цель – обучение генератора таким образом, чтобы порождённые  
им изображения отличались от источника исключительно в этом направлении. Авторы предлагают функционал  
потерь $\mathcal{L}_{direction}$, опирающийся на разницу $∆T$ эмбеддингов CLIP целевого и исходного текстов и разницу $∆I$ для  
изображений, порождаемых обучаемой версией генератора и исходной, которая не подвергается изменениям:
$$\\[2em]
∆T = E_{T} (t_{target} ) − E_{T}(t_{source} )
$$
$$\\[2em]
∆I = E_{I} (G_{train} (w)) − E_{I} (G_{frozen} (w))
$$
$$\\[2em]
\mathcal{L}_{direction} = 1 − \frac{∆I · ∆T}{|∆I| |∆T|}
$$

### Заморозка слоёв генератора

В работе авторы столкнулись с проблемами переобучения и схлопывания мод распределения (mode collapse)  
при достаточно долгой оптимизации, которая, однако, необходима при существенном изменении домена изображений  
(напр. из фото в скетч). Для борьбы с этим авторы опираются на ранние работы коллег и предлагают использовать  
ограничение обучаемых параметров, а именно – целых свёрточных слоёв генератора, оставляя лишь те, что наиболее  
сильно влияют на перевод изображения в целевой домен. Такой подход снижает сложность модели и уменьшает  
риск переобучения, к тому же ограничивая кол-во вычислений.

Для выбора наиболее значимых слоёв авторы предлагают использовать метод **Latent Optimization** из StyleCLIP,  
однако целью в этом случае является не сама оптимизация латента $\mathcal{W}^+$ – а наблюдение, какие его части (векторы) более  
всего изменились. Слои генератора, которые им соответствуют, далее обучаются, остальные "замораживаются".  
Также не подвергаются оптимизации **Mapping network** – полносвязная сеть в начале генератора, его части, отвечающие  
за аффинные преобразования, и слои **toRGB**.

### Обучение маппера

В случае сильных изменений домена авторы замечают, что обучившийся генератор иногда порождает изображения из  
обоих – нового и старого доменов. Чтобы этого избежать, они используют метод **Latent Mapper** из StyleCLIP,  
однако замечают в дополнении, что это может приводить к появлению характерных артефактов. Авторы видят их  
корреляцию с нормой векторов в пространстве CLIP. Для борьбы с ними предлагается использовать следующую  
регуляризацию на ограничение этой нормы:
$$\\[2em]
L_{norm} = |E_I(G(w)) - E_I(G(M(w)))|^2
$$

### ***
Это основные идеи статей. Далее их авторы приводят графики и примеры изображений, созданных с помощью  
обученных их способами моделей – вполне убедительные – и сравнивают результаты с другими методами,  
демонстрируя практическую ценность своих работ.